In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scipy.sparse import csc_matrix
import numpy as np
import torch

In [4]:
from go_metric.data_utils import TermDataset, TermSampler, BertSeqDataset
train_path = "/home/andrew/go_metric/data/go_bench"
train_dataset = BertSeqDataset.from_pickle(f"{train_path}/train.pkl")
val_dataset = BertSeqDataset.from_pickle(f"{train_path}/val.pkl")
test_dataset = BertSeqDataset.from_pickle(f"{train_path}/test.pkl")

In [5]:
import torch.utils.data as data
import torch
import numpy as np
from go_metric.data_utils import get_bert_seq_collator
collator = get_bert_seq_collator(max_length=1024)

term_dataset = TermDataset.from_pickle("/home/andrew/go_metric/data/go_bench/train.pkl")
labels = torch.BoolTensor(term_dataset.seq_dataset.labels.todense())
term_sampler = TermSampler(labels, 50000)

dl = data.DataLoader(term_dataset, sampler=term_sampler, batch_size=128, collate_fn=collator)